## Computational Chemistry for Experimentalists
## Module 6: Orbitals and Densities

Orbital hybridization is a central idea in general chemistry, and is put into practice in computational chemistry. However, understanding how to interpret orbital pictures isn't always easy. Here we'll show how different combinations of orbitals give different pictures of the same molecule.

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import math
import numpy
import matplotlib.pyplot as plt
from pyscf import gto,scf,dft,cc
from pyscf.tools import cubegen
import py3Dmol

## Example 1: Unambiguous Densities and Aesthetic Choices 

The stationary state wavefunction of nonrelativistic point-nucleus hydrogen atom can be determined *exactly*. However, different aesthetic choices in viewing that wavefunction can give very different pictures. Here we show the nearly exact hydrogen atom ground state using three different conventions. 

In [ ]:
m=gto.Mole(atom='H',spin=1,basis='aug-cc-pvqz')
m.build()
mf=scf.UHF(m)
mf.kernel()
cubegen.orbital(m, f'H_1s.cube', mf.mo_coeff[0,:,0],  nx=80, ny=80, nz=80)

In [ ]:
m2=Chem.MolFromSmiles('[H]')
p = py3Dmol.view(width=600,height=200,viewergrid=(1,3))
with open(f"H_1s.cube") as f:
    mo1 = f.read()
isovals=[0.3,0.1,0.02]
for i in range(3):
    p.addModel(Chem.MolToMolBlock(m2), 'sdf',viewer=(0,i))
    p.setStyle({'stick':{},'sphere':{"scale":0.1}},viewer=(0,i))
    p.addVolumetricData(mo1, "cube", {'isoval': isovals[i], 'color': "blue", 'opacity': 0.65},viewer=(0,i))
p.render()


## Example 2: Electrostatic Potentials 

Electron density isosurfaces can be kind of boring and featureless. Coloring the density isosurface with electrostatic potential gives a lot of additional information. However, interpreting these is an art in itself. 

We'll consider density isosurfaces for aniline and nitrobenzene, computed with HF/3-21G at the RDKit optimized geometries. This first block generates the 3D geometries and checks that they're reasonable. 

In [ ]:
ma=Chem.MolFromSmiles('c1ccc(N)cc1')
mn=Chem.MolFromSmiles('c1ccc(N(=O)=O)cc1')
ma2=Chem.AddHs(ma)
AllChem.EmbedMolecule(ma2)
AllChem.MMFFOptimizeMolecule(ma2)
mn2=Chem.AddHs(mn)
AllChem.EmbedMolecule(mn2)
AllChem.MMFFOptimizeMolecule(mn2)
ma3=Chem.MolToMolBlock(ma2)
mn3=Chem.MolToMolBlock(mn2)
p = py3Dmol.view(width=400,height=200,viewergrid=(1,2))
p.addModel(ma3, 'sdf',viewer=(0,0))
p.addModel(mn3, 'sdf',viewer=(0,1))
p.setStyle({'stick':{},'sphere':{"scale":0.1}})
p.render()

The next block generates the HF/3-21G orbitals of each molecule 

In [ ]:
elements = [atom.GetSymbol() for atom in ma2.GetAtoms()]
coordinates = ma2.GetConformer().GetPositions()
atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]
ma4 = gto.Mole(basis="3-21g")
ma4.atom = atoms
ma4.build();
mfa=scf.RHF(ma4)
mfa.kernel() 
elements = [atom.GetSymbol() for atom in mn2.GetAtoms()]
coordinates = mn2.GetConformer().GetPositions()
atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]
mn4 = gto.Mole(basis="3-21g")
mn4.atom = atoms
mn4.build();
mfn=scf.RHF(mn4)
mfn.kernel() 

Next block generates cubes of density and electrostatic potential 

In [ ]:
cubegen.density(ma4, f'aniline_density.cube', mfa.make_rdm1())
cubegen.mep(ma4, f'aniline_esp.cube', mfa.make_rdm1())
cubegen.density(mn4, f'nitrobenzene_density.cube', mfn.make_rdm1())
cubegen.mep(mn4, f'nitrobenzene_esp.cube', mfn.make_rdm1())

Next block visualizes the densities alone. This isn't very interesting! 

In [ ]:
with open(f"aniline_density.cube") as f:
    da = f.read()
with open(f"nitrobenzene_density.cube") as f:
    dn = f.read()
p = py3Dmol.view(width=400,height=200,viewergrid=(1,2))
p.addModel(ma3, 'sdf',viewer=(0,0))
p.addModel(mn3, 'sdf',viewer=(0,1))
p.addVolumetricData(da, "cube", {'isoval': 0.002, 'color': "gray", 'opacity': 0.75},viewer=(0,0))
p.addVolumetricData(dn, "cube", {'isoval': 0.002, 'color': "gray", 'opacity': 0.75},viewer=(0,1))
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.render()

In [ ]:
with open(f"aniline_esp.cube") as f:
    va = f.read()
with open(f"nitrobenzene_esp.cube") as f:
    vn = f.read()
p = py3Dmol.view(width=400,height=200,viewergrid=(1,2))
p.addModel(ma3, 'sdf',viewer=(0,0))
p.addModel(mn3, 'sdf',viewer=(0,1))
p.addVolumetricData(da, "cube", {'isoval': 0.002, 'opacity': 0.9,'voldata': va, 'volformat':'cube',
      'volscheme': {'gradient':'rwb', 'min':-.05, 'max':.05}},viewer=(0,0))
p.addVolumetricData(dn, "cube", {'isoval': 0.002, 'opacity': 0.9,'voldata': vn, 'volformat':'cube',
      'volscheme': {'gradient':'rwb', 'min':-.05, 'max':.05}},viewer=(0,1))
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.render()

### Example 3: Banana bonds 

In undergraduate chemistry, you learn that H2 has a single bond with covalent bond order 1, two electrons in a bonding orbital. He2 has two electrons in a bonding orbital and two electrons in an antibonding orbital,  covalent bond order 0. The He2 wavefunction can be written as two doubly-occupied atomic orbitals, or as doubly-occupied bonding and antibonding orbitals. Hybridizing the two doubly-occupied atomic orbitals does not change the energy. 

This can become confusing for ethylene. The ethylene wavefunction can be written as doubly occupied sigma and pi bonding orbitals, or as the two doubly occupied linear combinations sigma+pi and sigma-pi. Here you'll view these "banana bonds" and compare to the sigma and pi picture. Both pictures are equivalent, both pictures are equally valid, both are equally "real". 

This first block generates ethylene geometry and orbitals. 

In [ ]:
m=Chem.MolFromSmiles('C=C')
m2=Chem.AddHs(m)
AllChem.EmbedMolecule(m2)
AllChem.MMFFOptimizeMolecule(m2)
mb=Chem.MolToMolBlock(m2)
elements = [atom.GetSymbol() for atom in m2.GetAtoms()]
coordinates = m2.GetConformer().GetPositions()
atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]

pyscf_mole = gto.Mole(basis="3-21g")
pyscf_mole.atom = atoms
pyscf_mole.build();

mf=scf.RHF(pyscf_mole)
mf.kernel() 
p=py3Dmol.view(width=400,height=400)
p.addModel(mb,'sdf')
p.setStyle({'stick':{},'sphere':{"scale":0.3}})
p.zoomTo()
p.show()

This block computesthe highest-occupied orbital (pi bonding orbital), the third-highest-occupied bonding orbital (C-C sigma bonding orbital), and their linear combinations 

In [ ]:
cubegen.orbital(pyscf_mole, f'ethylene_pi.cube', mf.mo_coeff[:,7],  nx=60, ny=60, nz=60)
cubegen.orbital(pyscf_mole, f'ethylene_sigma.cube', mf.mo_coeff[:,5],  nx=60, ny=60, nz=60)
cubegen.orbital(pyscf_mole, f'ethylene_banana1.cube', (mf.mo_coeff[:,7]+mf.mo_coeff[:,5])/2**0.5,  nx=60, ny=60, nz=60)
cubegen.orbital(pyscf_mole, f'ethylene_banana2.cube', (mf.mo_coeff[:,7]-mf.mo_coeff[:,5])/2**0.5,  nx=60, ny=60, nz=60)

In [ ]:
p = py3Dmol.view(width=600,height=400,viewergrid=(2,2))
for i in range(2):
    for j in range(2):
        p.addModel(Chem.MolToMolBlock(m2), 'sdf',viewer=(i,j))
        p.setStyle({'stick':{},'sphere':{"scale":0.3}},viewer=(i,j))
with open(f"./ethylene_pi.cube") as f:
    pi = f.read()
p.addVolumetricData(pi, "cube", {'isoval': -0.02, 'color': "red", 'opacity': 0.65},viewer=(0,0))
p.addVolumetricData(pi, "cube", {'isoval': 0.02, 'color': "blue", 'opacity': 0.65},viewer=(0,0))
with open(f"./ethylene_sigma.cube") as f:
    sigma= f.read()
p.addVolumetricData(sigma, "cube", {'isoval': -0.02, 'color': "red", 'opacity': 0.65},viewer=(1,0))
p.addVolumetricData(sigma, "cube", {'isoval': 0.02, 'color': "blue", 'opacity': 0.65},viewer=(1,0))

with open(f"./ethylene_banana1.cube") as f:
    b1 = f.read()
p.addVolumetricData(b1, "cube", {'isoval': -0.02, 'color': "red", 'opacity': 0.65},viewer=(0,1))
p.addVolumetricData(b1, "cube", {'isoval': 0.02, 'color': "blue", 'opacity': 0.65},viewer=(0,1))
with open(f"./ethylene_banana2.cube") as f:
    b2= f.read()
p.addVolumetricData(b2, "cube", {'isoval': -0.02, 'color': "red", 'opacity': 0.65},viewer=(1,1))
p.addVolumetricData(b2, "cube", {'isoval': 0.02, 'color': "blue", 'opacity': 0.65},viewer=(1,1))
#p.zoomTo()
#p.update()
p.render()

Your assignment for this module has 3 parts. 

Part 1: The pictured density isosurfaces have little bumps and ridges on the surface. Explain why those are there, and do a calculation on aniline that reduces the size of the bumps on the surface. 

Part 2: Plot the electrostatic potential for CF3Br. Identify the sigma hole involved in halogen bonding.

Part 3: Plot the symmetry-adapted and equivalent lone pairs of water. A HF/3-21G calculation should provide equivalent lone pairs. For details, see https://en.wikipedia.org/wiki/Sigma-pi_and_equivalent-orbital_models 